### Mount Google Drive and login to HuggingFace

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip -q install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
import evaluate

In [5]:
dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize, batched=True)

README.md: 0.00B [00:00, ?B/s]

yelp_review_full/train-00000-of-00001.pa(…):   0%|          | 0.00/299M [00:00<?, ?B/s]

yelp_review_full/test-00000-of-00001.par(…):   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
small_train = dataset["train"].shuffle(seed=42).select(range(5000))
small_eval = dataset["test"].shuffle(seed=42).select(range(5000))

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)
"Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']"
"You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference."

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.'

In [9]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="yelp_review_classifier",
    eval_strategy="epoch",
    push_to_hub=False,
)

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.210800,1.107067,0.527800
2,0.895100,0.999073,0.571000
3,0.656900,1.187941,0.585600


TrainOutput(global_step=1875, training_loss=0.8326971923828125, metrics={'train_runtime': 802.0889, 'train_samples_per_second': 18.701, 'train_steps_per_second': 2.338, 'total_flos': 3946772136960000.0, 'train_loss': 0.8326971923828125, 'epoch': 3.0})

In [ ]:
!pip install trl

  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig

model_name = "gpt2"  # swap for a small instruct model if you have GPU
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token



In [ ]:
# Example preference dataset format: {prompt, chosen, rejected}
ds = load_dataset("Anthropic/hh-rlhf", split="train[:2%]")  # take a tiny slice to sanity check

# Map to fields DPOTrainer expects: "prompt", "chosen", "rejected"
def to_dpo(batch):
    return {
        "prompt": batch["prompt"],
        "chosen": batch["chosen"],
        "rejected": batch["rejected"],
    }
# print(ds)
# ds = ds.map(to_dpo, batched=True)
# ds = ds.select_columns(["prompt", "chosen", "rejected"])

In [ ]:
ds

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 3216
})